In [ ]:
! pip install datasets

In [ ]:
# import json
# import pandas as pd

# f = open('/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json')
# df = pd.DataFrame()
# for idx, line in enumerate(f):
#     if idx == 25000:
#         break
#     df = df.append({'title': json.loads(line)['title'], 'abstract': json.loads(line)['abstract']}, ignore_index=True)
    
# f.close()

In [ ]:
from datasets import load_dataset

# dataset = load_dataset('json', data_files='/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', split = '')
vals_ds = load_dataset('json', data_files='/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', split='train[0:2500]')
trains_ds = load_dataset('json', data_files='/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', split='train[2500:27500]')

In [ ]:
trains_ds

In [ ]:
from transformers import pipeline, AutoTokenizer, TFAutoModelForSeq2SeqLM

model_name ='t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
# summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    
    inputs = [prefix + doc for doc in examples["abstract"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [ ]:
train_tokenized = trains_ds.map(preprocess_function, batched=True)
test_tokenized = vals_ds.map(preprocess_function, batched=True)

In [ ]:
train_tokenized

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
tf_train_set = train_tokenized.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = test_tokenized.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
print(tf_train_set)
print(tf_test_set)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

In [ ]:
model.save_pretrained("./summarizer_model")

In [ ]:
from IPython.display import FileLink
FileLink(r'./summarizer_model/tf_model.h5')

In [ ]:
from datasets import load_metric

metric = load_metric("rouge")
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
test_ds = load_dataset('json', data_files='/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', split='train[30000:30100]')


In [ ]:
candidate_summaries = []

for i, text in enumerate(texts):
    if i % 100 == 0:
        print(i)
    input_ids = tokenizer(text, truncation=True, padding='longest', return_tensors="pt").input_ids.to('cpu')
    output = model.generate(input_ids, max_length=20)
    predictions = tokenizer.batch_decode(output, skip_special_tokens=True)
    candidate_summaries.append(predictions[0])


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

In [ ]:
summarizer(X_test[10064], max_length=20), y_test[10064]

In [ ]:
generated = [ summarizer(i, max_length=20) for i in X_test[:100] ]

In [ ]:
(generated[2]) #[0][0]['summary_text']

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
gen = [i[0]['summary_text'] for i in generated]
calc_rouge_scores(gen, y_test[:100])

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

In [ ]:
train_encodings = tokenizer(X_train,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(X_test,
                            truncation=True,
                            padding=True)